In [1]:
import numpy as np
import pandas as pd #data setini oluşturmak vs
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1" #145. satırdaki hatayı gidermek için geçmiş versinon kullanıyoruz
import tensorflow as tf
print(tf.__version__)
import pickle

2.13.0


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, GRU, Embedding

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [3]:
dataset = pd.read_csv('hepsiburada.csv') #pandas ile csv dosyasını okuduk

In [4]:
dataset

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
243492,1,fiyatına göre güzel
243493,1,Ürün kullanışlı iş görüyor fazlasıyla eşime al...
243494,1,"Hızlı Kargo, güzel ürün"
243495,1,telefon başarılı hızlı bir cihaz sadece beyaz...


In [5]:
target = dataset['Rating'].values.tolist() # etiketleri aldık

In [6]:
data = dataset['Review'].values.tolist() # verileri ayrı şekilde aldık

In [7]:
#BİR MODELİ EĞİTİRKEN VERİLERİ EĞİTİM VE TEST SETİ OLARAK AYIRIRIZ
#MODELİ EĞİTİM SETİ ÜZERİNDE EĞİTİRİZ VE EĞİTİM TAMAMLANDIKTAN SONRA
#TEST SETİ ÜZERİNDE MODELİ TEST EDERİZ BU ŞEKİLDE MODEL GERÇEKTEN HİÇ GÖRMEDİĞİ VERİLER ÜZERİNDE GERÇEKTEN BAŞARILIMI GÖRÜRÜZ
#EĞER EĞİTİM SETİ ÜZERİNDE BAŞARI VERİP TEST SETİ ÜZERİNDE BAŞARI SAĞLAMAZ İSE BİRŞEY ÖĞRENEMEMİŞ DEMEKTİR OVERFİTTİNG DENİR BUNA
#
cutoff = int(len(data) * 0.80) # yüzde 80 i aldık, test ve eğitim datası olarak ayırmak için
x_train, x_test = data[:cutoff], data[cutoff:] #ikiye ayırdık dataları aynı şeyi etiketler içinde yapıcaz
y_train, y_test = target[:cutoff], target[cutoff:]

In [8]:
x_train[500]

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım.bu yüzden tereddütsüz aldım . alacak olanlara öneririm'

In [9]:
y_train[500]

1

In [10]:
x_train[800] #modelimize eğitim aşamasında bu yorumun olumlu olduğunu öğreticez

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [11]:
y_train[800] #olumlu bir model

1

In [12]:
#her yorum tokenleştirilicek kelimelere ayrılacak ve sayı denk gelecek
#kelime haznemizde 10bin kelime alıcaz hepsini almıcaz datasetten
num_words = 10000 # en sık geçen 10bin kelime alınacak bunlar dışında kalanlar alınmayacak
#bu 10bin sınırını oyna ve modelin başarısına etkisini gör
tokenizer = Tokenizer(num_words=num_words) #kelime sınırını belirledik default u nan dır
#sınır belirlemek istemez ise nan yazarız

In [13]:
tokenizer.fit_on_texts(data) #yorum datamızı tokenizera aktardık reviewleeri tokenleştirk en cok kullanılan 10bin kelimeyi tokenleştirdi

In [14]:
#tokenizer.word_index  # HER KELİMEYE KARŞILIK GELEN BİR İNDEX BELİRLEDİK

In [15]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)  #eğitim setimizi tokenleştirdik ayırmış olduğumuz 80/20 olarak

In [16]:
x_train[800] #800üncü yoruma bakıyoruz öncelikle tokenleştirmeden önce haline bakıyoruz

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [17]:
print(x_train_tokens[800]) #her kelime sayılara çevrilmiş burada 10bin sınır belirlediğimiz için her kelimeye karşılık bir token bulunmayacak
#misal bilgisyarım kelimesi bu kelimenin karşılığı olmadığından 10bin sınırında olmadığından

[19, 341, 52, 194, 30, 54, 7992, 55, 209, 603, 7887, 3, 36, 114, 164, 479, 85, 1, 1682, 782, 30, 2, 380, 326, 3, 1, 81, 132, 562, 9, 10]


In [18]:
x_test_tokens = tokenizer.texts_to_sequences(x_test) #test cümlelerinide tokenleştirdik

In [19]:
#rnn ile oluşturduğumuz modellere ileri boyutlarda inputlar veririrz bizim datasetimizde farklı sayı kelimelerden oluşan cümleler bulunmakta farklı sayılarda olduğu için veremeyiz bu yüzden tüm yorumları belirli bir boyuta sabitleriz
#örn 50 kelimelik boyut belirledik diyelim ancak bir yorum 35 yorumsa 15 tane sıfır ekliyerek 50 yapılır,70 taneden oluşanda 50 ye düşürülür

In [20]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens] # list comprehension yapıyoruz tek satırda
num_tokens = np.array(num_tokens)

In [21]:
np.mean(num_tokens) #ortalama kaç token vardır diye baktık

20.744703220162876

In [22]:
np.max(num_tokens) # en fazla tokene baktık

295

In [23]:
np.argmax(num_tokens) # en uzun yorumun indexine baktık

21941

In [24]:
x_train[21941]# bu indexine baktık

'Özellikle bu kısma yazıyorum iyice okuyunuz,cihazın hızı çok iyi.Isınma normal boyutlarda.Kamerası elinizi sabit tutarsanız ve gündüz çok net gece ise çok net çekmez.Görüntülü konuşma özelliği yok ancak uygulama ile olur,anten çekimi gayet iyi.Parmak izi okuyucusu gayet hızlı bazen tuşu silmenizi istiyor ve okuyamıyor kirden vs...Ön kamera da tatmin edici,çekim esnasında ekran beyaz ışık vererek flaş görevi görür.Batarya bana 1 hafta gidiyor sık kullanımda ise 2-3 gün gidiyor.Hızlı şarj 1 saatte doluyor , şekilleri ele oturuyor ve şık bir görüntü var.Telefonu aldığım gün gittim ve ilk girdiğim yerden ekran koruyucu ve kılıf buldum.Kulaklık sesi çok net ve yüksek ancak kendi hoparlörü biraz zayıf sesi.Ekrana bakarken açık kalma özelliği yok.Diğer akıllı cihaz özellikleri %90 ı bu cihazda mevcut.Güç tasarrufu 2 ayrı modu var ve çok başarılı çalışıyor.4gb ram var genelde yarısı boş kalıyor.Bir de yeni cihazların çoğu titreşimi az ve sesi de az çıkıyor.Bu cihaz da içine dahil...Bu cihazla

In [25]:
max_tokens = np.mean(num_tokens) + 2* np.std(num_tokens) #std standard sapması
max_tokens = int(max_tokens) #float değeri int çevirdi
max_tokens #batch size sayılır
#ortalama bir sayı belirledik

59

In [26]:
np.sum(num_tokens < max_tokens) / len(num_tokens) # kaç tane 59 tane tokenden oluşur

0.9597982726686571

In [27]:
#yorumların yüzde 95 de 59 tokenden oluşan yorumlardan oluşur %5 59 fazla veya az olması sorun yatmaz

In [28]:
x_train_pad = pad_sequences(x_train_tokens, maxlen = max_tokens) #her yorumu eşit boyuda getiriyoruz 59 burada

In [29]:
x_test_pad = pad_sequences(x_test_tokens, maxlen = max_tokens) #bunada padding ekliyoruz

In [30]:
x_train_pad.shape #190 binden fazla yorum 59 a sabitlendi padding ile

(194797, 59)

In [31]:
x_test_pad.shape #48 binden fazla yorum 59 a sabitlendi padding ile

(48700, 59)

In [32]:
#artık verilerimiz hazır
np.array(x_train_tokens[800]) #800üncü yorumun normal haline bakalım ilk

array([  19,  341,   52,  194,   30,   54, 7992,   55,  209,  603, 7887,
          3,   36,  114,  164,  479,   85,    1, 1682,  782,   30,    2,
        380,  326,    3,    1,   81,  132,  562,    9,   10])

In [33]:
np.array(x_train_pad[21941]) #şindide padding eklenmiş  haline

array([ 537, 1804,  488, 2218,   75,  163,    2,  456,   13, 1308, 1865,
       3432,    5,  824, 7798,  836, 1640, 7838, 3344,   12, 1308,   31,
       3063,  299, 2704,  851, 1052,   39, 1490, 3436,  317, 1170,   75,
        537,   50,  327,  246, 1322,  120,    2, 3436, 7527,  199,  260,
        579, 6176, 1800,   13,   45,  808,  108,    3,  865,  749, 5840,
         50,  106,   76,  879])

In [34]:
np.array(x_train_pad[800])

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,   19,  341,   52,  194,   30,
         54, 7992,   55,  209,  603, 7887,    3,   36,  114,  164,  479,
         85,    1, 1682,  782,   30,    2,  380,  326,    3,    1,   81,
        132,  562,    9,   10])

In [35]:
idx = tokenizer.word_index #anahtar bölümünde kelimeler değer kısmında sayılar var
inverse_map = dict(zip(idx.values(),idx.keys())) #word indexi tersine çevirdik sayıdan kelimeyi bulmak için

In [36]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = ' '.join(words) # kelimeler aralarında boşluk bırakılarak yazılacak
    return text

In [37]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [38]:
tokens_to_string(x_train_tokens[800]) #sayısal tokenize edilmiş cümleyi vererek tokeni word_indeximizdeki sayılarla karşılaştırıp tekrak yazdırdık kelimeleri
# bazı kelimeler eksik gelecek cünkü 10bin sınırı vermiştik

'ürünü alalı 3 hafta kadar oldu aralıksız kullanıyorum sürekli açık durur ve ben günde yaklaşık 12 saat çok kullanırım şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim'

In [39]:
model = Sequential() #keras ile ardışık bir model oluşturucaz

In [40]:
embedding_size = 50 # her kelimeye karşılık gelen 50 uzunluğunda vector olacak

In [41]:
#biz burada kelime vektorlerini glove yada nltk hazır vektörler kullanmadan rastgele oluşturucaz ilk başta, fakat diğer derslerde hazır vektörleri görücez
#vektörleri rastgele oluşturucaz

In [42]:
model.add(Embedding(input_dim=num_words,
                   output_dim=embedding_size,
                   input_length = max_tokens, #mini batch
                   name='embedding_layer')) #100000x50 matrix oluşturduk giriş lenght 59
#bu ürün süper diye yorum gelirse bu kelimelerin karşığını 10bin kelimelerin içinden alıp aktarıcak bir sonraki layerin inputu oalcak

In [43]:
#RNN sinir ağımızı oluşturalım GRU kullanıcaz gradient descent hatasını çözücez
#3 tane 3 GRU layer ile eğitim yapılacak
#ilk grup layer ekleyelim model
model.add(GRU(units=16,return_sequences=True)) #CuDNNGRU ekran kartı ile kullanım için olabilir burada GRU neorunu 16 layerden output verecek
#True diyince sequences ın hepsi dönecek cünkü diğer layer tekrar gru olacak
model.add(GRU(units=8,return_sequences=True))# 8 Nörondan
model.add(GRU(units=4,return_sequences=False))  # false yaptık sonraki layer tek nörondan oluşacak
model.add(Dense(1, activation = 'sigmoid')) #output layeri sıfır-bir arası değer döndürür


In [44]:
optimizer = Adam(learning_rate=1e-3) #0.003 demektir

In [45]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding  (None, 59, 50)            500000    
 )                                                               
                                                                 
 gru (GRU)                   (None, 59, 16)            3264      
                                                                 
 gru_1 (GRU)                 (None, 59, 8)             624       
                                                                 
 gru_2 (GRU)                 (None, 4)                 168       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 504061 (1.92 MB)
Trainable params: 504061 (1.92 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [47]:
#59 tokene karşılık gelen 50 uzunluğunda ilk başta 3. layerde return seq yoktur dikkat et
#outputlar none yazıyor dikkat et batch oluyor bu
#modelimize 256 şar 256 şar beslicez verileri
#amac loss'u düşürüp accuracy'i arttırmak
#import datetime
#log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#tensorboard --logdir logs olarak kontrol edebilirsin terminalden
#hist = model.fit(np.array(x_train_pad),np.array(y_train),epochs=3, batch_size= 256,validation_data=(np.array(x_test_pad),np.array(y_test)),callbacks=[tensorboard_callback])
hist = model.fit(np.array(x_train_pad),np.array(y_train),epochs=3, batch_size= 256,validation_data=(np.array(x_test_pad),np.array(y_test))) #epochs ile 3 defa eğitilecek

Epoch 1/3
761/761 [==============================] - 46s 54ms/step - loss: 0.1921 - accuracy: 0.9516 - val_loss: 0.1355 - val_accuracy: 0.9560
Epoch 2/3
761/761 [==============================] - 41s 54ms/step - loss: 0.0956 - accuracy: 0.9700 - val_loss: 0.1199 - val_accuracy: 0.9578
Epoch 3/3
761/761 [==============================] - 40s 52ms/step - loss: 0.0710 - accuracy: 0.9784 - val_loss: 0.1245 - val_accuracy: 0.9590


In [48]:
history_dict = hist.history
print(history_dict.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [49]:
#eğitim esnasında test setini modele beslemedik
#o yüzden şimdi modeli test setiyle tamammen yabancı olduğu verileri ile test edicez
result = model.evaluate (np.array(x_test_pad),np.array(y_test))

1522/1522 [==============================] - 10s 6ms/step - loss: 0.1245 - accuracy: 0.9590


In [50]:
#test setimizdeki verilerle yüzde kaç isabet oranı doğruluk olduğuna bakıyoruz
result[1] #istabet oranı

0.958952784538269

In [51]:
#Video 4: bu derste eğittimiz modelde daha iyi fikir sahibi olmak için önce modelin hatalarını incelicez
#ve daha sonra kendi yazdığımız yorumları vererek modeli test edicez
#ilk önce modeli test setindeki ilk bin yorumda çalıştırıcaz daha sonra
#bu yorumlardan yanlış olanları bulup modelin yanlış tahmin ettiklerini bulucaz
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

32/32 [==============================] - 1s 6ms/step


In [52]:
#0.5 TEN BÜYÜKSE TAHMİN 1 DEĞİLSE 0 YAPICAZ BUDA BİZE OLUMLU OLUMSUZMU DİYECEK
cls_pred = np.array([1.0 if p> 0.5 else 0.0 for p in y_pred]) #1ler ve 0 lardan oluşan vector elde ettik

In [53]:
cls_true = np.array(y_test[0:1000])

In [54]:
#şimdi elimizdeki bu iki vektörü karşılaştırıcaz
#ve modleimizin yanlış tahminlerini bir vektör içinde saklıcaz
incorrect = np.where(cls_pred != cls_true)

In [55]:
incorrect = incorrect[0]

In [56]:
len(incorrect) #43 tanesi yanlış bilinmiş 1000 yorumdan

50

In [57]:
idx = incorrect[0]
idx

5

In [58]:
#5. yorumu yanlış bilmiş
text = x_test[idx]
text

'Ürün güzel ama teslimat çok geç yapıldı.'

In [59]:
#doğrusu tahmin etmesi zor bir yorum modelin hata yapması normal sayılabilir

In [60]:
y_pred[idx] #olumlu düşünüyor modelimiz aslında

0.8434444

In [61]:
cls_true[idx]# olumsuz olduğunu döndürdü

0

In [62]:
#şimdide kendi yorumlarımızı vererek modelin bu yorumları tahmin etmesini isteyelim
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"#olumlu başlanmış fakat sonra olumsuza dönmüş
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [63]:
#bu listeyi şuan direk modele veremeyiz öncelikle tokenleştirmemiz lazım

In [64]:
tokens = tokenizer.texts_to_sequences(texts)

In [65]:
tokens_pad = pad_sequences(tokens,maxlen = max_tokens)
tokens_pad.shape
tokens_pad[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        27,   1,  15,  94,  40,  29, 150])

In [66]:
model.predict(tokens_pad) #modelimiz verilen yorumlara göre olumlumu olumsuzmu bakıyor

1/1 [==============================] - 0s 31ms/step


array([[0.9944624 ],
       [0.98092973],
       [0.11616809],
       [0.987992  ],
       [0.12376229],
       [0.15142211],
       [0.9863958 ],
       [0.04938137],
       [0.99179727],
       [0.22978806]], dtype=float32)

In [67]:
#10 da 10 yaptı
#model.save("model.keras")
#model.save('model1.h5')
#model.save_weights('model1_weights.h5')
#with open('tokenizer.pickle', 'wb') as handle:
#    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [68]:
text11 = "kargo geç geldi ve ayrıca kargo paketi zarar görmüştü iade ettim"
text12 = "aldığım ürün bir türlü çalışmadı firmayı aradığımda bizden kaynaklı değil diyor o yüzden iade de kabul etmiyorlar"
text13 = "tam istediğim gibi bir ürün herkese tavsiye ederim"
text14 = "ürün güzel fakat sistemime uymadığı için iade etmek zorunda kaldım"
texts = [text11, text12, text13, text14]
tokens = tokenizer.texts_to_sequences(texts)
tokens_pad = pad_sequences(tokens,maxlen = max_tokens)
tokens_pad.shape
tokens_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  169,
          4,    2,  799, 6254, 5307, 9365, 9591, 2360,   50, 1238,   78,
        369,  677,   18, 2558])

In [69]:
model.predict(tokens_pad)

1/1 [==============================] - 0s 34ms/step


array([[0.13626058],
       [0.02175866],
       [0.99434865],
       [0.1090418 ]], dtype=float32)